In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 15.2 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data'
base_dir = '/content/drive/MyDrive/data'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
conv_base = tf.keras.applications.EfficientNetB1(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

27033600/27018416 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()

Model: "efficientnetb1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional)  (None, 7, 7, 1280)       6575239   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 1024)              64226304  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 70,809,743
Trainable params: 70,747,688
Non-trainable params: 62,055
_________________________________________________________________


In [ ]:
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 10

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)

label_map

Found 3604 images belonging to 8 classes.
Found 802 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [ ]:
train_stop = 3604//batch
test_stop = 401//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=[tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),
                       tfa.metrics.F1Score(num_classes=8),
                       tfa.metrics.FBetaScore(num_classes=8)])

In [ ]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=10)

Epoch 1/10
360/360 [==============================] - 1425s 4s/step - loss: 1.4971 - categorical_accuracy: 0.7685 - recall: 0.7599 - precision: 0.7805 - f1_score: 0.2003 - fbeta_score: 0.2003
Epoch 2/10
360/360 [==============================] - 404s 1s/step - loss: 1.5445 - categorical_accuracy: 0.7930 - recall: 0.7824 - precision: 0.7993 - f1_score: 0.2627 - fbeta_score: 0.2627
Epoch 3/10
360/360 [==============================] - 402s 1s/step - loss: 1.8875 - categorical_accuracy: 0.7835 - recall: 0.7710 - precision: 0.7908 - f1_score: 0.2807 - fbeta_score: 0.2807
Epoch 4/10
360/360 [==============================] - 399s 1s/step - loss: 2.0291 - categorical_accuracy: 0.8002 - recall: 0.7883 - precision: 0.8092 - f1_score: 0.2840 - fbeta_score: 0.2840
Epoch 5/10
360/360 [==============================] - 399s 1s/step - loss: 1.8142 - categorical_accuracy: 0.8108 - recall: 0.7986 - precision: 0.8170 - f1_score: 0.3013 - fbeta_score: 0.3013
Epoch 6/10
360/360 [========================

In [ ]:
test = model.evaluate(test_generator, steps=test_stop)

40/40 [==============================] - 162s 4s/step - loss: 1.4325 - categorical_accuracy: 0.8500 - recall: 0.8475 - precision: 0.8518 - f1_score: 0.4372 - fbeta_score: 0.4372


In [ ]:
model.save('/content/drive/MyDrive/dataset/ENetB1.h5')

In [ ]:
import pandas as pd

In [ ]:
df_marks = pd.DataFrame({'x_test': test_generator.filepaths,
     'y_test': test_generator.labels})

x_test = df_marks.drop(['y_test'],axis=1)
y_test = df_marks['y_test']



In [ ]:
pred = model.predict(test_generator)

In [ ]:
predicted_classes = np.argmax(pred, axis = 1)
print(predicted_classes)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3
 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 6 3 3 3 3 3 3 3 3 1
 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 2 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 1 1 3 3 3 3 3 3 3 3 3 3 3 6 2 3 3 3 3 3 3 3 5 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 2 3 2 3 2 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 6 3 3 1 

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
report = classification_report(y_test, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.07      0.04      0.05        70
           2       0.00      0.00      0.00        40
           3       0.81      0.93      0.87       650
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         8
           7       0.00      0.00      0.00         6

    accuracy                           0.76       802
   macro avg       0.11      0.12      0.12       802
weighted avg       0.66      0.76      0.71       802



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
